## Some Preprocessing

In [30]:
import numpy as np
import pandas as pd 
import feather as feather 
import random as random 
import os

In [22]:
%cd "Final DATA"

[WinError 2] The system cannot find the file specified: 'Final DATA'
D:\DATA SCIENCE\Movie_Recommendation_System\Final DATA


In [101]:
movies = feather.read_dataframe("feather/movies")
names = feather.read_dataframe("feather/names")

In [102]:
movies = movies.drop("Unnamed: 0",axis=1)

In [103]:
names = names.drop("Unnamed: 0",axis=1)

## Removing some more columns
- **weight_average** is the only metric to be kept for the ranking of the movies
- **number of votes** and **average rating** are removed

In [104]:
movies.columns

Index(['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear',
       'endYear', 'genres', 'averageRating', 'numVotes', 'directors',
       'weight_average', 'cast', 'director_name'],
      dtype='object')

In [105]:
movies = movies.drop(['averageRating', 'numVotes'],axis=1)

In [106]:
movies.loc[movies['titleType']=='tvEpisode']

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,genres,directors,weight_average,cast,director_name
680,tt4283088,tvEpisode,Battle of the Bastards,0,2016,\N,"Action,Adventure,Drama",nm0764601,8.942253,"['Liam Cunningham', 'Peter Dinklage', 'Kit Har...",['Miguel Sapochnik']
910,tt6027912,tvEpisode,The Long Night,0,2019,\N,"Action,Adventure,Drama",nm0764601,7.328648,"['Nikolaj Coster-Waldau', 'Peter Dinklage', 'K...",['Miguel Sapochnik']
1076,tt4283094,tvEpisode,The Winds of Winter,0,2016,\N,"Action,Adventure,Drama",nm0764601,8.708712,"['Lena Headey', 'Nikolaj Coster-Waldau', 'Pete...",['Miguel Sapochnik']
1160,tt2301451,tvEpisode,Ozymandias,0,2013,\N,"Crime,Drama,Thriller",nm0426059,8.718073,"['Bryan Cranston', 'Dean Norris', 'Aaron Paul'...",['Rian Johnson']
1298,tt6027916,tvEpisode,The Bells,0,2019,\N,"Action,Adventure,Drama",nm0764601,6.430178,"['Lena Headey', 'Nikolaj Coster-Waldau', 'Pete...",['Miguel Sapochnik']
...,...,...,...,...,...,...,...,...,...,...,...
38496,tt0582675,tvEpisode,I'm with the Band,0,1999,\N,"Comedy,Drama",nm0031976,7.206157,"['James Franco', 'Linda Cardellini', 'John Fra...",['Judd Apatow']
38497,tt0453555,tvEpisode,The Taming of the Shrew,0,2005,\N,"Comedy,Drama,Romance",nm0724047,7.202697,"['David Mitchell', 'Simon Chandler', 'Shirley ...",['David Richards']
38501,tt0702000,tvEpisode,Life's Too Short,0,2001,\N,"Comedy,Drama",nm0687964,7.207257,"['Frances Conroy', 'Peter Krause', 'Lauren Amb...",['Jeremy Podeswa']
38510,tt2476644,tvEpisode,Into the Deep,0,2012,\N,"Adventure,Fantasy,Romance",nm0881038,7.201484,"['Jennifer Morrison', 'Lana Parrilla', 'Ginnif...",['Ron Underwood']


### Removing Tv Episodes also
- TV episodes are not relevant for this recommendation system
- Removing the rows

In [107]:
ind = []
for k in movies.index:
    t = movies['titleType'][k]
    if t == 'tvEpisode':
        ind.append(k)
movies = movies.drop(ind,axis=0)

In [108]:
movies = movies.reset_index(drop=True)

In [109]:
movies.head()

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,genres,directors,weight_average,cast,director_name
0,tt0111161,movie,The Shawshank Redemption,0,1994,\N,Drama,nm0001104,9.209958,"['Morgan Freeman', 'Tim Robbins', 'William Sad...",['Frank Darabont']
1,tt0468569,movie,The Dark Knight,0,2008,\N,"Action,Crime,Drama",nm0634240,8.921540,"['Michael Caine', 'Christian Bale', 'Aaron Eck...",['Christopher Nolan']
2,tt1375666,movie,Inception,0,2010,\N,"Action,Adventure,Sci-Fi",nm0634240,8.722421,"['Leonardo DiCaprio', 'Ken Watanabe', 'Joseph ...",['Christopher Nolan']
3,tt0137523,movie,Fight Club,0,1999,\N,Drama,nm0000399,8.714213,"['Brad Pitt', 'Edward Norton', 'Zach Grenier',...",['David Fincher']
4,tt0944947,tvSeries,Game of Thrones,0,2011,2019,"Action,Adventure,Drama","nm0755261,nm0638354,nm0851930,nm0336241,nm0787...",9.181493,"['Iain Glen', 'Lena Headey', 'Nikolaj Coster-W...","['Daniel Sackheim', 'David Nutter', 'Alan Tayl..."


In [110]:
names.head()

,nconst,primaryName,knownForTitles
0,nm0005690,William K.L. Dickson,"tt0219560,tt1496763,tt1428455,tt0308254"
1,nm0721526,Émile Reynaud,"tt0413219,tt2184201,tt2184231,tt0000003"
2,nm5442194,Julien Pappé,tt0000003
3,nm0443482,Charles Kayser,tt0000005
4,nm0653042,John Ott,tt0000005


## Saving everything in lower case 

In [111]:
movies = movies.applymap(lambda s: s.lower() if type(s)==str else s)

In [115]:
names = names.applymap(lambda s: s.lower() if type(s)== str else s)

## Removing brackets for efficient split of names

In [112]:
movies['director_name'] = movies['director_name'].apply(lambda s:s[1:-1])
movies['cast'] = movies['cast'].apply(lambda s: s[1:-1])

In [116]:
movies['cast'][10].split(',')

["'laurence fishburne'",
 " 'keanu reeves'",
 " 'hugo weaving'",
 " 'carrie-anne moss'"]

## Finalised

In [117]:
movies.head()

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,genres,directors,weight_average,cast,director_name
0,tt0111161,movie,the shawshank redemption,0,1994,\n,drama,nm0001104,9.209958,"'morgan freeman', 'tim robbins', 'william sadl...",'frank darabont'
1,tt0468569,movie,the dark knight,0,2008,\n,"action,crime,drama",nm0634240,8.921540,"'michael caine', 'christian bale', 'aaron eckh...",'christopher nolan'
2,tt1375666,movie,inception,0,2010,\n,"action,adventure,sci-fi",nm0634240,8.722421,"'leonardo dicaprio', 'ken watanabe', 'joseph g...",'christopher nolan'
3,tt0137523,movie,fight club,0,1999,\n,drama,nm0000399,8.714213,"'brad pitt', 'edward norton', 'zach grenier', ...",'david fincher'
4,tt0944947,tvseries,game of thrones,0,2011,2019,"action,adventure,drama","nm0755261,nm0638354,nm0851930,nm0336241,nm0787...",9.181493,"'iain glen', 'lena headey', 'nikolaj coster-wa...","'daniel sackheim', 'david nutter', 'alan taylo..."


In [118]:
names.head()

,nconst,primaryName,knownForTitles
0,nm0005690,william k.l. dickson,"tt0219560,tt1496763,tt1428455,tt0308254"
1,nm0721526,émile reynaud,"tt0413219,tt2184201,tt2184231,tt0000003"
2,nm5442194,julien pappé,tt0000003
3,nm0443482,charles kayser,tt0000005
4,nm0653042,john ott,tt0000005


In [120]:
movies.to_feather("feather/movies")
names.to_feather("feather/names")

## Done.